# Smoker Dataset
## Module 10.13 Assignment 10: Final Project
Dec. 2024

Authors: Tyler Earps, Ryan Smith, Basil Mullings, & Ean Vandergraaf

### Abstract and Data Description

### Data Preparation and Feature Extraction

### Machine Learning Algorithm Preparation and Tuning

### Limitations, Future Work, and Conclusion